In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from scipy.io import loadmat
import pycwt as cwt
import seaborn as sns

# Function to calculate SI
def calculate_si_index(seizure_phases):
    seizure_phases_float = [float(phase) for phase in seizure_phases]
    complex_exponentials = np.exp(1j * np.array(seizure_phases_float))
    mean_resultant_vector = np.mean(complex_exponentials)
    SI = round(np.abs(mean_resultant_vector), 2)
    return SI

# Load the data from the MATLAB .mat file
mat_data = loadmat('All_counts.mat')
All_counts = mat_data['All_counts']

# Placeholder list to store significant peaks for each rat
significant_peaks = []

# Wavelet and filter variables
mother = cwt.Morlet(6)

# Define custom colors for lines and markers
line_color = '#1f77b4'  # blue
marker_color = '#d62728'  # red

# Create subplots for power spectrum
fig, axs = plt.subplots(2, 3, figsize=(15, 10))

# Create a separate figure for polar histograms
fig_polar, axs_polar = plt.subplots(2, 3, subplot_kw={'polar': True}, figsize=(12, 7))
colors = sns.color_palette("husl", 6)

# Iterate over each rat's data
for rat_idx, rat_data in enumerate(All_counts[0], start=1):
    # Extract data for the current rat
    rat_data = All_counts[0][rat_idx - 1]  # Subtract 1 to account for 0-based indexing
    rat_data = rat_data.T
    data_num = rat_data.flatten()

    num_frequencies = 100000  # Adjust as needed
    min_period = 4  # Minimum period in hours
    max_period = 40 * 24 / 4  # Maximum period in hours

    min_frequency = 1 / max_period
    max_frequency = 1 / min_period
    dt = 1

    freqs = np.linspace(min_frequency, max_frequency, num=num_frequencies)
    alpha, _, _ = cwt.ar1(data_num)
    wave, scales, freqs, coi, fft, fftfreqs = cwt.cwt(signal=data_num, dt=dt, wavelet=mother, freqs=freqs)

    power = np.abs(wave) ** 2
    period = 1 / freqs
    glbl_power = power.mean(axis=1)

    dof = data_num.size - scales
    var = data_num.std() ** 2
    glbl_signif, _ = cwt.significance(var, dt, scales, 1, alpha, significance_level=0.95, dof=dof, wavelet=mother)

    ind_peaks = find_peaks(glbl_power)[0]
    xpeaks = [period[i] for i in ind_peaks if glbl_power[i] > glbl_signif[i]]

    significant_peaks.append(xpeaks)

    ax = axs[(rat_idx - 1) // 3, (rat_idx - 1) % 3]
    ax.plot(period, glbl_power, color=line_color, label='Wavelet Power Spectrum')

    if xpeaks:
        powers = [glbl_power[i] for i in ind_peaks if glbl_power[i] > glbl_signif[i]]
        ax.scatter(xpeaks, powers, color=marker_color, marker='o', label='Significant Peaks')
    ax.plot(period, glbl_signif, 'k--', label='95% Significance Level')
    max_peak_period = xpeaks[np.argmax(powers)]

    ax.set_xlim(0, 240)
    ax.set_xlabel('Period (hours)')
    ax.set_ylabel('Power')
    ax.set_title(f'Rat T{rat_idx + 1}')
    ax.legend()

    ax_polar = axs_polar[(rat_idx - 1) // 3, (rat_idx - 1) % 3]

    seizure_timestamps = []
    for hour, count in enumerate(data_num):
        seizure_timestamps.extend([hour] * count)

    if xpeaks:
        max_peak_index = np.argmax(glbl_power[ind_peaks])
        cycle_length = max_peak_period

        phase_values = [(timestamp % cycle_length) * (2 * np.pi / cycle_length) for timestamp in seizure_timestamps]

        hist, bin_edges = np.histogram(phase_values, bins=24, range=(0, 2 * np.pi))

        ax_polar.bar(bin_edges[:-1], hist, color=colors[rat_idx - 1], edgecolor='black', fill=True, alpha=0.7, width=(2 * np.pi) / 24, align='edge')
        ax_polar.set_xticks([np.pi / 2, np.pi, 3 * np.pi / 2, 2 * np.pi])
        ax_polar.set_xticklabels(['Peak', 'Falling', 'Trough', 'Rising'], fontsize=11, color='grey')
        ax_polar.set_yticks([])
        ax_polar.tick_params(axis='both', pad=-4)
        ax_polar.grid(False)
        ax_polar.set_theta_zero_location('W')
        ax_polar.set_theta_direction(-1)
        ax_polar.set_rlabel_position(90)
        ax_polar.set_title(f'Rat T{rat_idx + 1}', fontsize=12, pad=20)

        bin_size_hours = cycle_length / 24
        # Annotate bin size
        ax_polar.text(-0.2, 1.2 * max(hist), f'Bin size: {bin_size_hours:.2f} hours', ha='center', fontsize=10, color='grey')

        SI = calculate_si_index(phase_values)
        # Annotate SI and period below the bin size
        ax_polar.text(0.2, 1.2 * max(hist), f'SI: {SI:.2f}\nPeriod: {cycle_length/24:.1f} days', ha='center', fontsize=13, color='black')

plt.tight_layout()
plt.savefig('polar_plot_hours.png', dpi=300)
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: 'All_counts.mat'

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from scipy.io import loadmat
import pycwt as cwt
import pandas as pd

# Load the data from the MATLAB .mat file
mat_data = loadmat('All_counts.mat')

# Extract the All_counts data
All_counts = mat_data['All_counts']

# Placeholder lists to store significant peaks and associated power for each rat
significant_peaks = []
peak_powers = []

# Wavelet and filter variables
mother = cwt.Morlet(6)

# Define custom colors for lines and markers
line_color = '#1f77b4'  # blue
marker_color = '#d62728'  # red

# Define the length of each segment and the overlap
segment_length = 360  # 240 hours
overlap = 1  # 24 hours
num_rats = 6

# Determine the total number of segments based on overlap
total_hours = len(All_counts[0][0].T.flatten())
num_segments = (total_hours - segment_length) // overlap + 1

for seg_idx in range(num_segments):
    segment_peaks = []
    segment_peak_powers = []

    for rat_idx, rat_data in enumerate(All_counts[0], start=1):
        # Extract data for the current rat
        rat_data = All_counts[0][rat_idx - 1]  # Subtract 1 to account for 0-based indexing
        rat_data = rat_data.T
        data_num = rat_data.flatten()

        # Get the current segment data with overlap
        start_idx = seg_idx * overlap
        end_idx = start_idx + segment_length
        segment_data = data_num[start_idx:end_idx]

        # Define the minimum and maximum periods
        min_period = 4  # Minimum period in hours
        max_period = 240  # Maximum period in hours (entire segment)

        # Define the corresponding frequencies
        min_frequency = 1 / max_period  # Maximum frequency (cycles per day)
        max_frequency = 1 / min_period  # Minimum frequency (cycles per day)
        num_frequencies = 1000  # Adjust as needed
        freqs = np.linspace(min_frequency, max_frequency, num=num_frequencies)
        dt = 1  # Sampling interval (1 hour)

        # Perform continuous wavelet transform
        alpha, _, _ = cwt.ar1(segment_data)  # Calculate lag-1 autocorrelation for significance
        wave, scales, freqs, coi, fft, fftfreqs = cwt.cwt(signal=segment_data, dt=dt, wavelet=mother, freqs=freqs)

        # Calculate wavelet power spectrum
        power = np.abs(wave) ** 2
        period = 1 / freqs
        glbl_power = power.mean(axis=1)

        dof = segment_data.size - scales  # Degrees of freedom
        var = segment_data.std() ** 2
        glbl_signif, _ = cwt.significance(var, dt, scales, 1, alpha, significance_level=0.95, dof=dof, wavelet=mother)

        # Find significant peaks in the global power spectrum
        ind_peaks = find_peaks(glbl_power)[0]
        significant_indices = [i for i in ind_peaks if glbl_power[i] > glbl_signif[i]]
        xpeaks = [period[i] for i in significant_indices]
        powers = [glbl_power[i] for i in significant_indices]

        segment_peaks.append(xpeaks)
        segment_peak_powers.append(powers)

    # Store significant peaks and their powers for the current segment across all rats
    significant_peaks.append(segment_peaks)
    peak_powers.append(segment_peak_powers)

# Convert the results to DataFrames and save to CSV files
for seg_idx, (segments_peaks, segments_powers) in enumerate(zip(significant_peaks, peak_powers), start=1):
    data = {
        'Rat': [],
        'Significant Period (hours)': [],
        'Power': []
    }

    for rat_idx, (peaks, powers) in enumerate(zip(segments_peaks, segments_powers), start=1):
        for peak, power in zip(peaks, powers):
            data['Rat'].append(f'Rat T{rat_idx+1}')
            data['Significant Period (hours)'].append(peak)
            data['Power'].append(power)

    df = pd.DataFrame(data)
    df.to_csv(f'significant_peaks_and_power_segment_{seg_idx}.csv', index=False)

    print(f'Segment {seg_idx} significant peaks and powers saved to CSV.')


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.gridspec as gridspec
import pandas as pd
from scipy.io import loadmat

# Load the data from the MATLAB .mat file
mat_data = loadmat('All_counts.mat')

# Extract the All_counts data
All_counts = mat_data['All_counts']

# Parameters
hours_per_day = 24
segment_length_hours = 15 * 24  # Length of each segment in hours
overlap_hours = 359            # Overlap between segments in hours
total_hours = 40 * 24          # Total duration in hours

# Automatically define segment times based on segment length and overlap
segment_times = {}
for seg_idx in range(1, total_hours - overlap_hours + 1):  # Segments starting from 1
    start_time = (seg_idx - 1) * (segment_length_hours - overlap_hours)
    end_time = start_time + segment_length_hours
    segment_times[seg_idx] = {'start': start_time, 'end': end_time}

# Load significant peaks and d_prime
def load_peaks_and_d_prime(segment_idx):
    filename = f'significant_peaks_and_power_segment_{segment_idx}.csv'
    print(f"Loading data from {filename}")
    df = pd.read_csv(filename)
    return df

# Define rat names explicitly from T2 to T7
rat_names = [f'Rat T{i}' for i in range(2, 8)]

# Collect all d_prime values to find the global min and max across all rats and segments
all_d_prime_values = []
for seg_idx in range(1, total_hours - overlap_hours + 1):
    for rat in rat_names:
        df = load_peaks_and_d_prime(seg_idx)
        rat_data = df[df['Rat'] == rat]
        all_d_prime_values.extend(rat_data['d_prime'])

# Get global min and max for d_prime
global_min_d_prime = min(all_d_prime_values)
global_max_d_prime = max(all_d_prime_values)

# Define a colormap with cool colors
cmap = plt.get_cmap('cool')

# Use PowerNorm for non-linear normalization (exponent < 1 to emphasize smaller values)
power_norm = mcolors.PowerNorm(gamma=0.5, vmin=global_min_d_prime, vmax=global_max_d_prime)

# Create a figure and GridSpec layout
n_rats = len(rat_names)
fig = plt.figure(figsize=(8, 6 * n_rats))
gs = gridspec.GridSpec(n_rats, 2, width_ratios=[10, 0.5], wspace=0.25)  # Adjusted width ratio for thicker color bar and reduced wspace

# Loop through each rat to create subplots and color bars
for i, rat in enumerate(rat_names):
    ax = plt.subplot(gs[i, 0])  # The main axis on the left (for the number of seizures)
    ax2 = ax.twinx()  # Create a twin axis for the right (for significant periods)

    print(f"Processing {rat}")

    # Plot the number of seizures histogram on the left y-axis
    seizures_per_hour = All_counts[0][i]  # Adjust index based on rat
    seizures_per_hour = seizures_per_hour.T.flatten()  # Flatten the array
    ax.fill_between(range(total_hours), 0, seizures_per_hour, color='grey', alpha=0.3)

    # Set larger font size for the left y-axis ticks (Number of Seizures)
    ax.tick_params(axis='y', labelsize=14)  # Increase font size for the left y-axis
    ax.set_ylabel('Number of Seizures (per hour)', fontsize=16)  # Label for the left y-axis

    # Set larger font size for the right y-axis ticks (Significant Periods)
    ax2.tick_params(axis='y', labelsize=14)  # Increase font size for the right y-axis
    ax2.set_ylabel('Significant Periods (days)', fontsize=16)  # Label for the right y-axis

    

    # Loop over each segment to plot the horizontal lines on the right y-axis (Significant Periods)
    for seg_idx, times in segment_times.items():
        df = load_peaks_and_d_prime(seg_idx)
        seg_data = df[df['Rat'] == rat]
        end_time = times['end']

        # Plot each peak cycle as a horizontal line with color based on d_prime
        for _, row in seg_data.iterrows():
            peak = row['Significant Period (hours)'] / 24  # Convert hours to days
            d_prime = row['d_prime']
            color = cmap(power_norm(d_prime))  # Apply power-law normalization to emphasize smaller d_prime
            ax2.hlines(peak, end_time - 1, end_time, colors=[color], linewidth=7)

    # Customize the plot appearance
    ax.set_xlabel('Time (days)', fontsize=16)
    ax.set_xlim(0, total_hours)
    ax2.set_ylim(0, 240 / 24)  # Set y-axis limit to 10 days (240 hours / 24) for Significant Periods
    
    # Set larger font size for x and y tick labels on the x-axis
    ax.tick_params(axis='x', labelsize=14)  # X-axis tick size
    
    x_ticks = np.arange(0, total_hours + 1, 24 * 5)  # Keep ticks as days
    day_labels = [str(int(x / 24)) for x in x_ticks]
    ax.set_xticks(x_ticks)
    ax.set_xticklabels(day_labels, fontsize=14)  # Larger font size for x-tick labels

    ax.set_title(f'{rat}', fontsize=16)
    ax.grid(True)  # Keep grid lines on the left y-axis
# Add a black horizontal line lower on the y-axis (e.g., at 10 days)
    ax.hlines(y=0, xmin=1, xmax=15*24, color='black', linestyle='--', linewidth=3, label='')
    # Create a color bar for this subplot
    cbar_ax = plt.subplot(gs[i, 1])
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=power_norm)  # Use power normalization
    sm.set_array([])
    cbar = plt.colorbar(sm, cax=cbar_ax, orientation='vertical')
    cbar.ax.tick_params(labelsize=14)  # Set larger font size for the color bar ticks
   # cbar.set_label("d'", fontsize=14)  # Label the color bar as d'

# Adjust layout with more space between subplots and color bar
plt.subplots_adjust(wspace=0.25, hspace=0.3)  # Reduce wspace slightly to make the distance smaller

# Show and save the plot for all rats
plt.tight_layout()
plt.show()
fig.savefig('peak_cycles_over_time_all_rats_d_prime_power_norm_swapped_axes_with_spacing_thicker_colorbar.png', dpi=300)  # Save with a general name for all rats
